# Jupyter notebook sample

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix

from keras.datasets import mnist
import keras

%matplotlib inline
plt.style.use('default')

### Loading MNIST data

Effortlessly view, navigate, sort, and filter data. Create charts and access essential data insights, including descriptive statistics and missing values – all without writing a single line of code.

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# separate x_train in X_train and X_val, same for y_train
X_train=x_train[0:50000] / 255 #divide by 255 so that they are in range 0 to 1
Y_train=keras.utils.to_categorical(y_train[0:50000],10) # one-hot encoding

X_val=x_train[50000:60000] / 255
Y_val=keras.utils.to_categorical(y_train[50000:60000],10)

X_test=x_test / 255
Y_test=keras.utils.to_categorical(y_test,10)

#print(X_test[0])

del x_train, y_train, x_test, y_test

X_train=np.reshape(X_train, (X_train.shape[0],28,28,1))
X_val=np.reshape(X_val, (X_val.shape[0],28,28,1))
X_test=np.reshape(X_test, (X_test.shape[0],28,28,1))

#print(X_train.shape)
#print(X_val.shape)
#print(X_test.shape)
#print(Y_train.shape)
#print(Y_val.shape)
#print(Y_test.shape)

### Visualization in IDE

Create graphs and visualizations that match your chosen color scheme.

In [ ]:
# relabel
def relabel(labels, threshold=5):
    relabeled = []
    for label in labels:
        if np.argmax(label) >= threshold:
            relabeled.append([1,0])
        else:
            relabeled.append([0,1])
    return np.array(relabeled,dtype=np.float32)

Y_train_relabeled = relabel(Y_train)
Y_val_relabeled = relabel(Y_val)
Y_test_relabeled = relabel(Y_test)

In [ ]:
base_model = tf.keras.models.load_model("../aufgabe5/e6.keras")
base_model.pop()
base_model.trainable = False
base_model.summary()
tf.keras.utils.plot_model(base_model)

Der folgende Code funktioniert nicht:
```python
inputs = base_model.inputs
x = base_model(inputs, training=False)
output = Dense(2, activation='softmax')(x)

model = Model(inputs, output)
```

Auch der hier funktioniert nicht, selber Fehler:

```python
inputs = base_model.inputs
x = base_model.output
output = Dense(2, activation='softmax')(x)
```

Er führt im evaluierungsschritt und training zu

> ValueError: The metric has not been built.


Nur das hier funktioniert:
```python
inputs = base_model.inputs
x = base_model(inputs, training=False)
output = Dense(2, activation='softmax')(base_model.layers[-1].output)
```

In [ ]:
from keras.src.layers import Dense
from keras import Model

inputs = base_model.inputs
output = Dense(2, activation='softmax')(base_model.layers[-1].output)

model = Model(inputs=inputs, outputs=output)
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
learning_rate = 0.0001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()

In [ ]:
keras.config.disable_traceback_filtering()
t = model.evaluate(x = X_test, y = Y_test_relabeled)
print(t)

In [ ]:
epochs = 10
history = model.fit(X_train, Y_train_relabeled, batch_size=128, epochs=epochs, validation_data=(X_val, Y_val_relabeled))

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], linestyle='-.')
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], linestyle='-.')
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')

In [ ]:
pred = model.predict(X_test)
print(confusion_matrix(np.argmax(Y_test_relabeled,axis=1), np.argmax(pred, axis=1)))
acc_fc_orig = np.sum(np.argmax(Y_test_relabeled,axis=1) == np.argmax(pred, axis=1)) / len(pred)
print("Acc_fc_orig_flat = ", acc_fc_orig)